In [161]:
# Broker_vendor.py
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle


# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

# Replace these values with your Oracle database connection details
username = 'apps'
password = 'apps'
host = 'csebsd2db.cswg.com' #EBSD2
port = '1521'
service_name = 'csebsd2_int' #EBSD2

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
connection = cx_Oracle.connect(username, password, dsn)


In [173]:
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle
from psycopg2 import sql


# PostgreSQL connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Oracle connection parameters
oracle_username = 'apps'
oracle_password = 'apps'
oracle_host = 'csebsd2db.cswg.com'
oracle_port = '1521'
oracle_service_name = 'csebsd2_int'



post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

# XML file path
xml_file_path = 'D:\\W\\RELEX\\RELEX\\Broker and Vendor\\Inbound Files\\VENDOR.xml'
# xml_file_path = 'D:\\W\\RELEX\\RELEX\\Broker and Vendor\\Inbound Files\\BROKER.xml'


# Open the XML file
with open(xml_file_path, 'r', encoding='utf-8') as file:
    xml_data = file.read()


# Table name
table_name = "cspoms.XXPO_ORCL2CSPOMS_VEND_STG".upper()

# Get the nextval for request_id
nextval_request_id = "SELECT nextval('cspoms.xxpo_cs_relex_run_id_seq')"
post_cur.execute(nextval_request_id)
request_id = post_cur.fetchone()[0]
#         post_conn.commit()
print(f"Request ID: {request_id}")

        # Get the current timestamp
current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current Timestamp: {current_timestamp}")
# Split the XML data into lines
lines = xml_data.splitlines()

# Regular expression pattern to match the elements
pattern = re.compile(r"<([^>]+)>([^<]+)</\1>")
tag_to_column = {
    'BICEPSVENDOR': 'BIC_VENDOR_NUMBER',
    'BIC_VENDOR_FACILITY':'BIC_VENDOR_FACILITY',
    'BUYERNUMBER': 'BIC_BUYER_NUMBER',
    'CRPLEVEL': 'BIC_CRP_LEVEL',
    'STATUS': 'BIC_STATUS',
    'PORECOMMENDEDIND': 'BIC_PO_RECOMMENDED_IND',
    'AFETYPE1': 'BIC_AFE_TYPE1',
    'AFEFAXNUMBER1': 'BIC_AFE_FAX_NUMBER',
    'AFECONTACT1': 'BIC_AFE_CONTACT',
    'MINIMUMQUANTITY': 'BIC_MINIMUM_QUANTITY',
    'MINIMUMTYPE': 'BIC_MINIMUM_TYPE',
    'BKTQUANTITY1': 'BIC_BRACKET_QUANTITY',
    'CURRENTBKTNUMBER': 'BIC_CURRENT_BKT_NUMBER',
    'CURRENTBKTQUANTITY': 'BIC_CURRENT_BKT_QUANTITY',
    'CURRENTBKTTYPE': 'BIC_CURRENT_BKT_TYPE',
    'MAXIMUMQUANTITY': 'BIC_MAXIMUM_QUANTITY',
    'MAXIMUMTYPE': 'BIC_MAXIMUM_TYPE',
    'ORDERINTERVALWEEKS': 'BIC_ORDER_INTERVAL_WEEKS',
    'LEADTIMESTATEDWEEKS': 'BIC_LEAD_TIME_STATED_WEEKS',
    'FIXEDREVIEWDAY1': 'BIC_FIXED_REVIEW_DAY1',
    'FIXEDREVIEWDAY2': 'BIC_FIXED_REVIEW_DAY2',
    'FIXEDREVIEWDAY3': 'BIC_FIXED_REVIEW_DAY3',
    'FIXEDREVIEWDAY4': 'BIC_FIXED_REVIEW_DAY4',
    'FIXEDREVIEWDAY5': 'BIC_FIXED_REVIEW_DAY5',
    'FIXEDREVIEWDAY6': 'BIC_FIXED_REVIEW_DAY6',
    'FIXEDREVIEWDAY7': 'BIC_FIXED_REVIEW_DAY7',
    'TARGETSERVICELEVEL': 'BIC_TARGET_SERVICE_LEVEL',
    'NAME': 'BIC_NAME',
    'ADDRESS1': 'BIC_ADDRESS_1',
    'CITY': 'BIC_CITY',
    'STATE': 'BIC_STATE',
    'ZIP': 'BIC_ZIP',
    'PHONE': 'BIC_PHONE',
    'CONTACT': 'BIC_CONTACT',
    'TRANSFERMATCHTYPE': 'BIC_TRANSFER_MATCH_TYPE',
    'PAYABLEVENDORNBR': 'BIC_PAYABLE_VENDOR_NBR',
    'LEADTIMEWEEKSTYPE': 'BIC_LEAD_TIME_WEEKS_TYPE',
    'DUNSREMITNO': 'BIC_DUNS_REMIT_NO',
    'DUNSREMITSUFFIX': 'BIC_DUNS_REMIT_SUFFIX',
    'FACILITYSHIPTO': 'BIC_FACILITY_SHIP_TO',
    'PRESCHED': 'BIC_PRE_SCHED_FLAG',
    'BROKERNUMBER': 'BIC_BROKER_NUMBER',
    'FREIGHTALLOW': 'BIC_FREIGHT_ALLOW',
    'FREIGHTALLOWTYPE': 'BIC_FREIGHT_ALLOW_TYPE',
    'TERMSPERCENT': 'BIC_TERMS_PERCENT',
    'TERMSDAYS': 'BIC_TERMS_DAYS',
    'TERMSNETDAYS': 'BIC_TERMS_NET_DAYS',
    'TERMSBASE': 'BIC_TERMS_BASE',
    'BILLFORSHORTEDDEALSFLAG': 'BIC_BILL_FOR_SHORT_DEALS_FLAG',
    'TRANSITDAYS': 'BIC_TRANSIT_DAYS',
    'DUNSNO': 'BIC_DUNS_NO',
    '2NDMINQUANTITY': 'BIC_2ND_MIN_QUANTITY',
    '2NDMINTYPE': 'BIC_2ND_MIN_TYPE',
    '2NDMAXQUANTITY': 'BIC_2ND_MAX_QUANTITY',
    '2NDMAXTYPE': 'BIC_2ND_MAX_TYPE',
    'FLAGPREPAID': 'BIC_FLAG_PREPAID',
    'FLAGPREPAYANDADD': 'BIC_FLAG_PREPAY_AND_ADD',
    'FLAGFREIGHTBILL': 'BIC_FLAG_FREIGHT_BILL',
    'FLAGBACKHAUL': 'BIC_FLAG_BACKHAUL',
    'FLAGTRUCK': 'BIC_FLAG_TRUCK',
    'FLAGRAIL': 'BIC_FLAG_RAIL',
    'BACKHAUL': 'BIC_BACKHAUL',
    'BACKHAULTYPE': 'BIC_BACKHAUL_TYPE',
    'FREIGHTBILL': 'BIC_FREIGHTBILL',
    'ORDERSEQUENCE': 'BIC_ORDER_SEQUENCE',
    'SSAALLOWPERCENT': 'BIC_SSA_ALLOW_PERCENT',
    'SSAALLOWBASIS': 'BIC_SSA_ALLOW_BASIS',
    'SSAALLOWACCOUNT': 'BIC_SSA_ALLOW_ACCOUNT',
    'TRANSACTIONID': 'BIC_TRANSACTION_ID',
    'COUNTRY': 'BIC_COUNTRY',
    'ORDERFILTER': 'BIC_FILTER_VENDOR_FLAG',
    'LBLIMIT': 'BIC_LOAD_BLDG_LIMIT_FLAG',
    'WAREHOUSECODE': 'BIC_WAREHOUSE_CODE',
    'FLAGEXTBCKHL': 'BIC_EXTERNAL_BACKHAUL'
}
# print('tag_to_column : ',len(tag_to_column))
# Loop through each line in the XML data
for line in lines:
    # Skip empty lines or lines containing only whitespace
    if not line.strip():
        continue

    # Define a dictionary to store values
    element_values = {}

    # Find all matches in the text data
    matches = re.findall(pattern, line)

    # Store the matches in the dictionary
    for tag, value in matches:
        element_values[tag] = value.strip()
    # Assuming 'BICEPSVENDOR' is present in element_values
    if 'BICEPSVENDOR' in element_values:
        bicepsvendor_value = element_values['BICEPSVENDOR']

        # Check if the value has at least 3 characters
        if len(bicepsvendor_value) >= 3:
            # Extract the first 3 digits
            facility_value = bicepsvendor_value[:3]

            # Assign the first 3 digits to a new key
            element_values['BIC_VENDOR_FACILITY'] = facility_value

            # Update 'BICEPSVENDOR' with the remaining value
            element_values['BICEPSVENDOR'] = bicepsvendor_value[3:]
#     print('element_values:############',element_values)
    # Now 'BIC_VENDOR_FACILITY' and updated 'BICEPSVENDOR' are present in element_values
    # Replace column names with the ones from tag_to_column
    mapped_values = {tag_to_column[key]: value for key, value in element_values.items() if key in tag_to_column}
    # Include additional columns in the mapping
    mapped_values['REQUEST_ID'] = request_id
    mapped_values['CREATION_DATE'] = current_timestamp
    mapped_values['LAST_UPDATE_DATE'] = current_timestamp
    mapped_values['CREATED_BY'] = -1
    mapped_values['LAST_UPDATED_BY'] = -1
#     print('mapped_values:########0',mapped_values)
    # Generate SQL INSERT statement
    columns_str = ",".join(mapped_values.keys())
    values_str = ",".join(f"'{v}'" for v in mapped_values.values())

    insert_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({values_str});"
#     print(insert_query)

# Establish a database connection and execute the INSERT statement

    post_cur.execute(insert_query)
#     print(insert_query)


#     print(f"Record inserted: {mapped_values}")

try:

    # Retrieve the current timestamp from the database
    profile_update_query = "SELECT CURRENT_TIMESTAMP"
    post_cur.execute(profile_update_query)
    current_timestamp = post_cur.fetchone()[0]

    # Oracle connection
    oracle_dsn = cx_Oracle.makedsn(oracle_host, oracle_port, service_name=oracle_service_name)
    oracle_conn = cx_Oracle.connect(oracle_username, oracle_password, dsn=oracle_dsn)
    oracle_cur = oracle_conn.cursor()
    


    update_bic_columns_for_file_data(post_cur)
    post_cur.execute("SELECT * FROM XXPO_ORCL2CSPOMS_VEND_STG WHERE PROCESS_STATUS ='U'")
    upsert_rows = post_cur.fetchall()
    column_names = [desc[0] for desc in post_cur.description]
    upsert_df = pd.DataFrame(upsert_rows, columns=column_names)
    upsert_df['vendor_type'] = upsert_df['vendor_type'].str[0]    
    print(upsert_df)
    print('before')
    upsert_vendor_master_and_childs_test(upsert_df,post_cur)
    print('done')
    post_conn.commit()

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close PostgreSQL connection and cursor
    try:
        if post_cur:
            post_cur.close()
        if post_conn:
            post_conn.close()
    except Exception as e:
        print(f"Error closing the PostgreSQL connection: {e}")

    # Close Oracle connection and cursor
    try:
        if oracle_cur:
            oracle_cur.close()
        if oracle_conn:
            oracle_conn.close()
    except Exception as e:
        print(f"Error closing the Oracle connection: {e}")


Request ID: 204
Current Timestamp: 2024-01-25 15:23:58
@@@@@  START of update_bic_columns_for_file_data @@@@@
  bic_vendor_number  bic_vendor_facility
0            445932                  251
1            447709                  252

            SELECT *
            FROM APPS.XXAP_VENDOR_MASTER
            WHERE (VENDOR_NBR, VENDOR_HOME_LOC_DEST) IN (
                ('445932', '251'), ('447709', '252')
            ) LIMIT 999
        
An error occurred: Execution failed on sql '
            SELECT *
            FROM APPS.XXAP_VENDOR_MASTER
            WHERE (VENDOR_NBR, VENDOR_HOME_LOC_DEST) IN (
                ('445932', '251'), ('447709', '252')
            ) LIMIT 999
        ': ORA-00933: SQL command not properly ended


In [170]:
upsert_df

,stg_seq_id,bic_vendor_facility,bic_vendor_number,bic_buyer_number,bic_crp_level,bic_status,bic_po_recommended_ind,bic_afe_type1,bic_afe_fax_number,bic_afe_contact,...,pickup_day_6,pickup_day_7,eoq_cap_days,whse_ship_to,ship_with_vendor,rank_percent_override,po_date_change_rsn_flag,pullbks_available,hawaii_trans_method,record_type
0,3,251,445932,J8,,A,B,,,,...,None,None,None,None,000000,None,None,None,None,None
1,5,250,417384,,,A,B,,999 999 9999,FAX NUMBER,...,None,None,None,None,None,None,None,None,None,None
2,4,252,447709,J8,,A,B,,,,...,None,None,None,None,000000,None,None,None,None,None
3,1,252,008571,ZZ,,A,B,,,,...,None,None,None,None,None,None,None,None,None,None
4,2,905,008571,ZZ,,A,B,,,,...,None,None,None,None,None,None,None,None,None,None
5,6,251,445932,J8,,A,B,,,,...,None,None,None,None,000000,None,None,None,None,None
6,8,251,445932,J8,,A,B,,,,...,None,None,None,None,000000,None,None,None,None,None
7,10,251,445932,J8,,A,B,,,,...,None,None,None,None,000000,None,None,None,None,None
8,12,251,445932,J8,,A,B,,,,...,None,None,None,None,000000,None,None,None,None,None
9,14,251,445932,J8,,A,B,,,,...,None,None,None,None,000000,None,None,None,None,None


In [171]:
def update_bic_columns_for_file_data(post_cur):
    print('@@@@@  START of update_bic_columns_for_file_data @@@@@')
    def update_postgres_from_oracle(oracle_df, post_cur, include_facility_condition=True):
#       oracle_df = oracle_df.fillna('')
        # Replace 'None' with None for all columns in the DataFrame
        for index, row in oracle_df.iterrows():
            # Create the update query dynamically for all columns


            update_query = f"""
                UPDATE XXPO_ORCL2CSPOMS_VEND_STG
                SET 
        PROCESS_STATUS = 'U',
        VENDOR_SITE_ID = %s, 
        VENDOR_NBR = %s,
        VENDOR_NAME = %s, 
        VENDOR_START_DATE = %s,
        BUYER_VENDOR = %s,
        BUYER_VENDOR_NAME = %s,
        VENDOR_HOME_LOC_DEST = %s,
        SHIP_TO_LOCATION_NAME = %s,
        VENDOR_STATUS = %s,
        AP_VENDOR_NBR = %s,
        AP_VENDOR_NAME = %s,
        BICEPS_BUYER_CODE = %s,
        BUYER_NAME = %s,
        VENDOR_FOODLINK_FLG = %s,
        ADDR_LINE1 = %s,
        ADDR_LINE2 = %s,
        ADDR_LINE3 = %s,
        CITY = %s,
        STATE = %s,
        ZIP_CODE = %s,
        COUNTRY = %s,
        PHONE_NBR = %s,
        VENDOR_CONTACT_NAME = %s,
        BILL_TO_LOCATION_NAME = %s,
        BUILD_TO_DAYS_SUPPLY = %s,
        DAYS_TO_BUILD_TO = %s,
        USE_CYCLE_STOCK_AS_MIN = %s,
        USE_FAC_WHSE_SS = %s,
        MAIN_FACILITY = %s,
        LEAD_TIME_OVRD_DAYS_1 = %s,
        LEAD_TIME_OVRD_DAYS_2 = %s,
        LEAD_TIME_OVRD_DAYS_3 = %s,
        LEAD_TIME_OVRD_DAYS_4 = %s,
        LEAD_TIME_OVRD_DAYS_5 = %s,
        LEAD_TIME_OVRD_DAYS_6 = %s,
        LEAD_TIME_OVRD_DAYS_7 = %s,
        APPLY_RATIONING = %s,
        OP_DAYS_1ST_RUN_1 = %s,
        OP_DAYS_1ST_RUN_2 = %s,
        OP_DAYS_1ST_RUN_3 = %s,
        OP_DAYS_1ST_RUN_4 = %s,
        OP_DAYS_1ST_RUN_5 = %s,
        OP_DAYS_1ST_RUN_6 = %s,
        OP_DAYS_1ST_RUN_7 = %s,
        OP_DAYS_2ND_RUN_1 = %s,
        OP_DAYS_2ND_RUN_2 = %s,
        OP_DAYS_2ND_RUN_3 = %s,
        OP_DAYS_2ND_RUN_4 = %s,
        OP_DAYS_2ND_RUN_5 = %s,
        OP_DAYS_2ND_RUN_6 = %s,
        OP_DAYS_2ND_RUN_7 = %s,
        SHIP_WITH_VENDOR = %s,
        VENDOR_TYPE = %s,
        CONNECTING_VENDOR = %s,
        TRANSIT_DAYS = %s,
        LAG_DAYS = %s,
        LEAD_TIME_CALENDAR_IND = %s,
        VENDOR_LEAD_DAYS = %s,
        VENDOR_MIN_ORDER_SIZE = %s,
        VENDOR_ORDER_UNIT = %s,
        VENDOR_MAX_ORDER_SIZE = %s,
        VENDOR_MIN_ORDER2_SIZE = %s,
        VENDOR_ORDER2_UNIT = %s,
        VENDOR_MAX_ORDER2_SIZE = %s,
        VENDOR_DLVR_MONDAY_FLG = %s,
        VENDOR_DLVR_TUESDAY_FLG = %s,
        VENDOR_DLVR_WEDNESDAY_FLG = %s,
        VENDOR_DLVR_THURSDAY_FLG = %s,
        VENDOR_DLVR_FRIDAY_FLG = %s,
        VENDOR_DLVR_SATURDAY_FLG = %s,
        VENDOR_DLVR_SUNDAY_FLG = %s,
        VENDOR_SERVICE_LEVEL = %s,
        DAYS_TO_BUY_OVERRIDE = %s,
        LOAD_BLDG_LIMIT_FLG = %s,
        FILTER_VENDOR_FLG = %s,
        VENDOR_DISC_PCT = %s,
        VENDOR_DISCOUNT_DAYS_NBR = %s,
        VENDOR_NET_DAYS = %s,
        VENDOR_CONTINUITY_REPLEN_FLG = %s
                    
                    

                WHERE
                    bic_vendor_number = LPAD('{row['VENDOR_NBR']}', 6, '0')
                    {'AND bic_vendor_facility = ' + str(row['VENDOR_HOME_LOC_DEST']) if include_facility_condition else ''}
            """
            values = (
                row['VENDOR_SITE_ID'],
                row['VENDOR_NBR'],
                row['VENDOR_NAME'],
                row['VENDOR_START_DATE'],
                row['BUYER_VENDOR'],
                row['BUYER_VENDOR_NAME'],
                row['VENDOR_HOME_LOC_DEST'],
                row['SHIP_TO_LOCATION_NAME'],
                row['VENDOR_STATUS'],
                row['AP_VENDOR_NBR'],
                row['AP_VENDOR_NAME'],
                row['BICEPS_BUYER_CODE'],
                row['BUYER_NAME'],
                row['VENDOR_FOODLINK_FLG'],
                row['ADDR_LINE1'],
                row['ADDR_LINE2'],
                row['ADDR_LINE3'],
                row['CITY'],
                row['STATE'],
                row['ZIP_CODE'],
                row['COUNTRY'],
                row['PHONE_NBR'],
                row['VENDOR_CONTACT_NAME'],
                row['BILL_TO_LOCATION_NAME'],
                row['BUILD_TO_DAYS_SUPPLY'],
                row['DAYS_TO_BUILD_TO'],
                row['USE_CYCLE_STOCK_AS_MIN'],
                row['USE_FAC_WHSE_SS'],
                row['MAIN_FACILITY'],
                row['LEAD_TIME_OVRD_DAYS_1'],
                row['LEAD_TIME_OVRD_DAYS_2'],
                row['LEAD_TIME_OVRD_DAYS_3'],
                row['LEAD_TIME_OVRD_DAYS_4'],
                row['LEAD_TIME_OVRD_DAYS_5'],
                row['LEAD_TIME_OVRD_DAYS_6'],
                row['LEAD_TIME_OVRD_DAYS_7'],
                row['APPLY_RATIONING'],
                row['OP_DAYS_1ST_RUN_1'],
                row['OP_DAYS_1ST_RUN_2'],
                row['OP_DAYS_1ST_RUN_3'],
                row['OP_DAYS_1ST_RUN_4'],
                row['OP_DAYS_1ST_RUN_5'],
                row['OP_DAYS_1ST_RUN_6'],
                row['OP_DAYS_1ST_RUN_7'],
                row['OP_DAYS_2ND_RUN_1'],
                row['OP_DAYS_2ND_RUN_2'],
                row['OP_DAYS_2ND_RUN_3'],
                row['OP_DAYS_2ND_RUN_4'],
                row['OP_DAYS_2ND_RUN_5'],
                row['OP_DAYS_2ND_RUN_6'],
                row['OP_DAYS_2ND_RUN_7'],
                row['SHIP_WITH_VENDOR'],
                row['VENDOR_TYPE'],
                row['CONNECTING_VENDOR'],
                row['TRANSIT_DAYS'],
                row['LAG_DAYS'],
                row['LEAD_TIME_CALENDAR_IND'],
                row['VENDOR_LEAD_DAYS'],
                row['VENDOR_MIN_ORDER_SIZE'],
                row['VENDOR_ORDER_UNIT'],
                row['VENDOR_MAX_ORDER_SIZE'],
                row['VENDOR_MIN_ORDER2_SIZE'],
                row['VENDOR_ORDER2_UNIT'],
                row['VENDOR_MAX_ORDER2_SIZE'],
                row['VENDOR_DLVR_MONDAY_FLG'],
                row['VENDOR_DLVR_TUESDAY_FLG'],
                row['VENDOR_DLVR_WEDNESDAY_FLG'],
                row['VENDOR_DLVR_THURSDAY_FLG'],
                row['VENDOR_DLVR_FRIDAY_FLG'],
                row['VENDOR_DLVR_SATURDAY_FLG'],
                row['VENDOR_DLVR_SUNDAY_FLG'],
                row['VENDOR_SERVICE_LEVEL'],
                row['DAYS_TO_BUY_OVERRIDE'],
                row['LOAD_BLDG_LIMIT_FLG'],
                row['FILTER_VENDOR_FLG'],
                row['VENDOR_DISC_PCT'],
                row['VENDOR_DISCOUNT_DAYS_NBR'],
                row['VENDOR_NET_DAYS'],
                row['VENDOR_CONTINUITY_REPLEN_FLG']
                )
            
            print(update_query)
            # Execute the update query in PostgreSQL
            post_cur.execute(update_query,values)
        print("Update successful.")
        
            
    # PostgreSQL Query
    post_cur.execute("SELECT DISTINCT bic_vendor_number, bic_vendor_facility FROM XXPO_ORCL2CSPOMS_VEND_STG WHERE PROCESS_STATUS ='N' LIMIT 999")
    result_rows = post_cur.fetchall()

    # Check if result_rows has any rows
    if result_rows:
        column_names = [desc[0] for desc in post_cur.description]
        stg_df = pd.DataFrame(result_rows, columns=column_names)
        print(stg_df)

        # Convert 'bic_vendor_number' to numeric before creating the Oracle query
        # Assuming 'VENDOR_NBR' is the column for which you want to create the IN clause
        query = f"""
            SELECT *
            FROM APPS.XXAP_VENDOR_MASTER
            WHERE (VENDOR_NBR, VENDOR_HOME_LOC_DEST) IN (
                {", ".join(f"('{v}', '{d}')" for v, d in zip(map(str, pd.to_numeric(stg_df['bic_vendor_number'], errors='coerce')), stg_df['bic_vendor_facility']))}
            ) 
        """
        print(query)

        # Oracle Query
        oracle_df = pd.read_sql(query, oracle_conn)

        if not oracle_df.empty:
            print(oracle_df)
            # Example usage:
            update_postgres_from_oracle(oracle_df, post_cur)

    else:
        print("Join with VENDOR_NUMBER AND VENDOR_FACILITY rows no match found in XXAP_VENDOR_MASTER. CHECKING for VENDOR_NBR further processing....")

    post_cur.execute("SELECT DISTINCT bic_vendor_number FROM XXPO_ORCL2CSPOMS_VEND_STG WHERE PROCESS_STATUS ='N'")
    result_rows2 = post_cur.fetchall()

    # Check if result_rows2 has any rows
    if result_rows2:
        column_names2 = [desc[0] for desc in post_cur.description]
        stg_vn_df = pd.DataFrame(result_rows2, columns=column_names2)
        print(stg_vn_df)

        # Assuming 'VENDOR_NBR' is the column for which you want to create the IN clause
        query_vn = f"""
            SELECT *
            FROM APPS.XXAP_VENDOR_MASTER
            WHERE VENDOR_NBR IN ({", ".join(map(lambda v: f"'{v}'", pd.to_numeric(stg_vn_df['bic_vendor_number'], errors='coerce')))})
        """

        # Optional: If you want to filter out NULL values from 'bic_vendor_number'
        # query_vn += " AND VENDOR_NBR IS NOT NULL"

        print(query_vn)

        # Oracle Query
        oracle_vn_df = pd.read_sql(query_vn, oracle_conn)
        if not oracle_vn_df.empty:
            print(oracle_vn_df)

            # Example usage:
            update_postgres_from_oracle(oracle_vn_df, post_cur, False)
    else:
        print("No rows found while joining only with VENDOR_NBR in XXAP_VENDOR_MASTER, further processing...")

    error_update_query = f"""
                    UPDATE XXPO_ORCL2CSPOMS_VEND_STG
                    SET 
                        PROCESS_STATUS = 'E',
                        ERROR_MESSAGE = 'bic_vendor_number nor bic_vendor_facility found in Oracle Denorm XXAP_VENDOR_MASTER'
                    WHERE
                        PROCESS_STATUS = 'N'
                """

    print(error_update_query)
    # Execute the update query in PostgreSQL
    post_cur.execute(error_update_query)
    print('@@@@@  END of update_bic_columns_for_file_data @@@@@')

In [128]:
oracle_vn_df
stg_vn_df
oracle_df
stg_df
upsert_df
upsert_df[['bic_vendor_number','vendor_site_id','bic_vendor_facility','connecting_vendor','vendor_name','vendor_type',
'request_id','main_facility','build_to_days_supply','days_to_build_to',
'bic_freight_allow_type','bic_freight_allow','vendor_continuity_replen_flg'
]]

,bic_vendor_number,vendor_site_id,bic_vendor_facility,connecting_vendor,vendor_name,vendor_type,request_id,main_facility,build_to_days_supply,days_to_build_to,bic_freight_allow_type,bic_freight_allow,vendor_continuity_replen_flg
0,417384,3411105,250,None,GSOTC TEST,Buyer Vendor,167,None,None,None,,0.0000,None
1,447709,3276057,252,None,None,Buyer Vendor,167,0,N,0,,0.0000,None
2,008571,3218150,252,None,McELRONE SALES INC,Broker,167,None,None,None,,0.0000,None
3,008571,3218150,905,None,McELRONE SALES INC,Broker,167,None,None,None,,0.0000,None


In [146]:


def upsert_vendor_master_and_childs_test(merged_df,cur):
    # Iterate over DataFrame rows for upsert
    for index, row in merged_df.iterrows():
        # cur.execute("SELECT nextval('cs_vendor_id_seq')")
        # next_sequence_value = cur.fetchone()[0]
        
        # print(next_sequence_value)
        cur.execute(
        f'''INSERT INTO cspoms.XXPO_VENDOR_MASTER (
            CS_VENDOR_ID,
            VENDOR_NUMBER,
            VENDOR_FACILITY,
            CS_VEND_SITE_ID,
            CONNECTING_VENDOR,
            VENDOR_NAME,
            VENDOR_TYPE,


            REQUEST_ID,
            CREATION_DATE,
            LAST_UPDATE_DATE,
            CREATED_BY,
            LAST_UPDATED_BY
        ) 
        VALUES (
            
             nextval('cs_vendor_id_seq'),
            %s,
            %s,
            %s,
            %s,
            %s,
            %s,
            
            %s,
            current_timestamp,
            current_timestamp,
            '-1',
            '-1'


        ) 
        ON CONFLICT (VENDOR_NUMBER, VENDOR_FACILITY) DO UPDATE SET 
            VENDOR_NUMBER = EXCLUDED.VENDOR_NUMBER,
            VENDOR_FACILITY = EXCLUDED.VENDOR_FACILITY,
            CS_VEND_SITE_ID = EXCLUDED.CS_VEND_SITE_ID,
            CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR,
            VENDOR_NAME = EXCLUDED.VENDOR_NAME,
            VENDOR_TYPE = EXCLUDED.VENDOR_TYPE,

            REQUEST_ID = EXCLUDED.REQUEST_ID,
            LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
            LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY

            RETURNING CS_VENDOR_ID ''',(row['bic_vendor_number'],row['bic_vendor_facility'],row['vendor_site_id'],row['connecting_vendor'],
            row['vendor_name'],row['vendor_type'],row['request_id'])
    )


        # Fetch the generated C1 value
        generated_c1 = cur.fetchone()[0]

        # If C1 was generated or updated, insert/update other tables
        if generated_c1:
            print('insert if')
            cur.execute(
        f'''
        INSERT INTO XXPO_VENDOR_ORDER_POINT (
            CS_VENDOR_ID,
            MAIN_FACILITY,
            BUILD_TO_DAYS_SUPPLY,
            DAYS_TO_BUILD_TO,

            REQUEST_ID,
            CREATION_DATE,
            LAST_UPDATE_DATE,
            CREATED_BY,
            LAST_UPDATED_BY
        )
        VALUES (
            {generated_c1},
            %s,
            %s,
            %s,

            %s,
            current_timestamp,
            current_timestamp,
            '-1',
            '-1'
        )
        ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
            MAIN_FACILITY = EXCLUDED.MAIN_FACILITY,
            BUILD_TO_DAYS_SUPPLY = EXCLUDED.BUILD_TO_DAYS_SUPPLY,
            DAYS_TO_BUILD_TO = EXCLUDED.DAYS_TO_BUILD_TO,

            REQUEST_ID = EXCLUDED.REQUEST_ID,
            LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
            LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
        ''',(row['main_facility'],row['build_to_days_supply'],row['days_to_build_to'],row['request_id']
        )
    )


            cur.execute(
                f'''
                INSERT INTO XXPO_VENDOR_ORDER_INFO (
                    CS_VENDOR_ID,
                    VENDOR_NUMBER,
                    CONNECTING_VENDOR,

             
                    REQUEST_ID,
                    CREATION_DATE,
                    LAST_UPDATE_DATE,
                    CREATED_BY,
                    LAST_UPDATED_BY
                )
                VALUES (
                    {generated_c1},
                    %s,
                    %s,

                    %s,
                    current_timestamp,
                    current_timestamp,
                    '-1',
                    '-1'
                )
                ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
                    VENDOR_NUMBER = EXCLUDED.VENDOR_NUMBER,
                    CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR,

                    REQUEST_ID = EXCLUDED.REQUEST_ID,
                    LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
                    LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
                ''',(row['bic_vendor_number'],row['connecting_vendor'],row['request_id']
                )
            )



            cur.execute(
        f'''
        INSERT INTO XXPO_VENDOR_COSTING_INFO (
            CS_VENDOR_ID,
            FREIGHT_ALLOW_BUY_RATE,
            FREIGHT_ALLOW_BUY_RATE_TYPE,
            
            REQUEST_ID,
            CREATION_DATE,
            LAST_UPDATE_DATE,
            CREATED_BY,
            LAST_UPDATED_BY
        )
        VALUES (
            {generated_c1},
            %s,
            %s,
            
            %s,
            current_timestamp,
            current_timestamp,
            '-1',
            '-1'
        )
        ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
            FREIGHT_ALLOW_BUY_RATE = EXCLUDED.FREIGHT_ALLOW_BUY_RATE,
            FREIGHT_ALLOW_BUY_RATE_TYPE = EXCLUDED.FREIGHT_ALLOW_BUY_RATE_TYPE,

            REQUEST_ID = EXCLUDED.REQUEST_ID,
            LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
            LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
        ''',(row['bic_freight_allow'],row['bic_freight_allow_type'],row['request_id'],
        ))


            cur.execute(f'''INSERT INTO XXPO_VENDOR_EDI_INFO (CS_VENDOR_ID,
                VENDOR_CONTINUITY_REPLEN_FLG,
                REQUEST_ID,
                CREATION_DATE,
                LAST_UPDATE_DATE,
                CREATED_BY,
                LAST_UPDATED_BY
                ) 
                VALUES (
                {generated_c1},
                %s,
                %s,
                current_timestamp,
                current_timestamp,
                '-1',
                '-1') 
                ON CONFLICT (CS_VENDOR_ID) 
                DO UPDATE SET VENDOR_CONTINUITY_REPLEN_FLG = EXCLUDED.VENDOR_CONTINUITY_REPLEN_FLG,
                REQUEST_ID = EXCLUDED.REQUEST_ID,
                LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
                LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
                ''',
                (
                row['vendor_continuity_replen_flg'],
                row['request_id']
                ))

In [10]:
# Create a query based on stg_df values using IN clause
query = f"""
    SELECT *
    FROM APPS.XXAP_VENDOR_MASTER
    WHERE (VENDOR_NBR, VENDOR_HOME_LOC_DEST) IN (
        {", ".join(f"('{v}', '{d}')" for v, d in zip(stg_df['bic_vendor_number'], stg_df['bic_vendor_facility']))}
    )
"""
query

"\n    SELECT *\n    FROM APPS.XXAP_VENDOR_MASTER\n    WHERE (VENDOR_NBR, VENDOR_HOME_LOC_DEST) IN (\n        ('447709', '252'), ('417384', '250'), ('8571', '252'), ('8571', '905'), ('445932', '251')\n    )\n"

In [12]:
# Create a query based on stg_df values using CTE
cte_query = f"""
    WITH stg_data AS (
        SELECT *
        FROM (SELECT {", ".join(f"'{v}', '{d}'" for v, d in zip(stg_df['bic_vendor_number'], stg_df['bic_vendor_facility']))} FROM dual) t(VENDOR_NBR, VENDOR_HOME_LOC_DEST)
    )
    SELECT AV.*
    FROM APPS.XXAP_VENDOR_MASTER AV
    JOIN stg_data SD ON AV.VENDOR_NBR = SD.VENDOR_NBR AND AV.VENDOR_HOME_LOC_DEST = SD.VENDOR_HOME_LOC_DEST
"""
oracle_df = pd.read_sql(cte_query, oracle_conn)
print(oracle_df[['VENDOR_NBR', 'VENDOR_HOME_LOC_DEST']])

InterfaceError: not connected

In [123]:
for i in upsert_df.columns:
    print(i)

stg_seq_id
bic_vendor_facility
bic_vendor_number
bic_buyer_number
bic_crp_level
bic_status
bic_po_recommended_ind
bic_afe_type1
bic_afe_fax_number
bic_afe_contact
bic_minimum_quantity
bic_minimum_quantity_sign
bic_minimum_type
bic_bracket_quantity
bic_bracket_quantity_sign
bic_current_bkt_number
bic_current_bkt_quantity
bic_current_bkt_quantity_sign
bic_current_bkt_type
bic_maximum_quantity
bic_maximum_quantity_sign
bic_maximum_type
bic_order_interval_weeks
bic_order_interval_weeks_sign
bic_lead_time_stated_weeks
bic_lead_time_stated_wks_sign
bic_fixed_review_day1
bic_fixed_review_day2
bic_fixed_review_day3
bic_fixed_review_day4
bic_fixed_review_day5
bic_fixed_review_day6
bic_fixed_review_day7
bic_target_service_level
bic_target_service_level_sign
bic_name
bic_address_1
bic_city
bic_state
bic_zip
bic_phone
bic_contact
bic_transfer_match_type
bic_payable_vendor_nbr
bic_lead_time_weeks_type
bic_duns_remit_no
bic_duns_remit_suffix
bic_facility_ship_to
bic_pre_sched_flag
bic_broker_number
